In [ ]:
````xml
<VSCode.Cell language="markdown">
# 🎤 Agent Vocal IA - Démo Simplifiée pour Colab
## 100% Local | Temps Réel | Multi-Domaines

---

### 📋 Ce que fait cet agent

- 🎙️ **Écoute** vos questions vocales (français)
- 🧠 **Comprend** le domaine (maths, physique, anglais)
- 📚 **Recherche** dans sa base de connaissances (RAG)
- 💬 **Répond** de manière pédagogique
- 🔊 **Parle** la réponse en français

### ⚡ Installation : ~10 minutes

**⚠️ IMPORTANT** : Activez le GPU !
- Menu : `Runtime → Change runtime type → GPU (T4)`

---
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 📦 Étape 1 : Installation des Dépendances (~3 min)
</VSCode.Cell>

<VSCode.Cell language="python">
import sys
import subprocess

print("🚀 Installation des dépendances...")
print("=" * 70)

# Installation système
!apt-get update -qq
!apt-get install -y -qq portaudio19-dev ffmpeg espeak-ng > /dev/null 2>&1

# Installation Python (sans pipecat pour simplifier)
packages = [
    "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",
    "openai-whisper",
    "faster-whisper",
    "langchain",
    "langchain-community",
    "chromadb",
    "sentence-transformers",
    "faiss-cpu",
    "soundfile",
    "gradio",
    "pydub",
    "pyttsx3"
]

for pkg in packages:
    print(f"Installing {pkg.split()[0]}...")
    subprocess.run(f"{sys.executable} -m pip install -q {pkg}", shell=True)

print("\n✅ Dépendances installées!")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🔍 Étape 2 : Vérification GPU
</VSCode.Cell>

<VSCode.Cell language="python">
import torch

print("🔍 Vérification du GPU...")
print("=" * 70)

if torch.cuda.is_available():
    print(f"✅ GPU : {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    DEVICE = "cuda"
else:
    print("⚠️ Pas de GPU - utilisation CPU (plus lent)")
    DEVICE = "cpu"

print(f"🎯 Device : {DEVICE}")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 📥 Étape 3 : Clonage du Projet
</VSCode.Cell>

<VSCode.Cell language="python">
import os

print("📥 Clonage du projet...")
print("=" * 70)

if os.path.exists("intelligence_lab_agent_vocal"):
    !rm -rf intelligence_lab_agent_vocal

!git clone -b pipecat-local-colab https://github.com/Romainmlt123/agent-vocal-IA-Rag-Agentique.git intelligence_lab_agent_vocal

os.chdir("intelligence_lab_agent_vocal")
sys.path.insert(0, os.getcwd())

print(f"✅ Projet cloné!")
print(f"📁 Dossier : {os.getcwd()}")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🤖 Étape 4 : Installation Ollama + Modèle LLM (~3 min)
</VSCode.Cell>

<VSCode.Cell language="python">
import subprocess
import time

print("🤖 Installation Ollama...")
print("=" * 70)

!curl -fsSL https://ollama.com/install.sh | sh

print("\n🚀 Démarrage serveur Ollama...")
ollama_process = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)

print("✅ Serveur démarré!")

print("\n📥 Téléchargement modèle Qwen2 1.5B (~900MB)...")
!ollama pull qwen2:1.5b

print("\n✅ Modèle LLM prêt!")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🎙️ Étape 5 : Téléchargement Whisper (~2 min)
</VSCode.Cell>

<VSCode.Cell language="python">
from faster_whisper import WhisperModel

print("🎙️ Téléchargement Whisper base...")
print("=" * 70)

whisper_model = WhisperModel(
    "base",
    device=DEVICE,
    compute_type="float16" if DEVICE == "cuda" else "int8"
)

print("✅ Whisper prêt!")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 📚 Étape 6 : Construction Index RAG (~1 min)
</VSCode.Cell>

<VSCode.Cell language="python">
print("📚 Construction index RAG...")
print("=" * 70)

# Utiliser le script legacy qui fonctionne
result = subprocess.run(
    [sys.executable, "-m", "src.legacy.rag_build"],
    capture_output=True,
    text=True,
    cwd="/content/intelligence_lab_agent_vocal"
)

if result.returncode == 0:
    print("✅ Index RAG construits!")
    print("   • data/maths/index.faiss")
    print("   • data/physique/index.faiss")
    print("   • data/anglais/index.faiss")
else:
    print("⚠️ Erreur:")
    print(result.stderr)
    # Continuer quand même

print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🎨 Étape 7 : Initialisation Composants
</VSCode.Cell>

<VSCode.Cell language="python">
print("🎨 Initialisation composants...")
print("=" * 70)

# Importer l'ancienne architecture (plus stable pour Colab)
from src.legacy.router import Router
from src.legacy.rag import RAG
from src.legacy.llm import LLM
from src.legacy.asr import ASR
from src.legacy.tts import TTS
from src.legacy.orchestrator import Orchestrator

print("⚙️ Router...")
router = Router()

print("⚙️ RAG...")
rag = RAG()

print("⚙️ LLM (Ollama)...")
llm = LLM(model_name="qwen2:1.5b")

print("⚙️ ASR (Whisper)...")
asr = ASR(model_size="base", device=DEVICE)

print("⚙️ TTS...")
tts = TTS()

print("⚙️ Orchestrator...")
orchestrator = Orchestrator(
    router=router,
    rag=rag,
    llm=llm,
    asr=asr,
    tts=tts
)

print("\n✅ Tous les composants initialisés!")
print("🔄 Audio → Whisper → Router → RAG → Ollama → TTS → Audio")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🎭 Étape 8 : Interface Gradio
</VSCode.Cell>

<VSCode.Cell language="python">
import gradio as gr
import numpy as np

def process_audio(audio_input):
    """Traite l'audio et retourne la réponse"""
    if audio_input is None:
        return "❌ Aucun audio", "", "", None, ""
    
    try:
        sample_rate, audio_data = audio_input
        
        # Convertir en mono si stereo
        if len(audio_data.shape) > 1:
            audio_data = audio_data.mean(axis=1)
        
        # Normaliser
        audio_data = audio_data.astype(np.float32)
        if audio_data.max() > 1.0:
            audio_data = audio_data / 32768.0
        
        # Traiter
        result = orchestrator.process_question(audio_data, sample_rate)
        
        # Sources
        sources_text = "\n\n".join([
            f"📄 **{src['title']}** (score: {src['score']:.2f})\n{src['content'][:150]}..."
            for src in result.get('sources', [])
        ]) or "Aucune source"
        
        return (
            result['transcription'],
            result['subject'],
            result['response'],
            (result['sample_rate'], result['audio']),
            sources_text
        )
        
    except Exception as e:
        return f"❌ Erreur: {str(e)}", "", "", None, ""


def process_text(text_input):
    """Traite une question texte (pour debug/test)"""
    if not text_input:
        return "", "", "", None, ""
    
    try:
        # Simuler le traitement audio avec du texte
        result = orchestrator.process_text_question(text_input)
        
        sources_text = "\n\n".join([
            f"📄 **{src['title']}** (score: {src['score']:.2f})\n{src['content'][:150]}..."
            for src in result.get('sources', [])
        ]) or "Aucune source"
        
        return (
            text_input,
            result['subject'],
            result['response'],
            (result['sample_rate'], result['audio']) if 'audio' in result else None,
            sources_text
        )
    except Exception as e:
        return text_input, "", f"❌ Erreur: {str(e)}", None, ""


# Interface Gradio
with gr.Blocks(title="🎤 Agent Vocal IA", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎤 Agent Vocal IA avec RAG Agentique
    ### 100% Local | Temps Réel | Multi-Domaines
    
    **Mode d'emploi** :
    1. **Onglet Audio** : Enregistrez votre question vocalement
    2. **Onglet Texte** : Tapez votre question (plus rapide pour tester)
    3. Attendez la réponse (~2-3 secondes)
    4. Écoutez l'audio généré
    
    **Domaines** : Maths 🧮 | Physique ⚛️ | Anglais 🇬🇧
    """)
    
    with gr.Tabs():
        # Onglet Audio
        with gr.Tab("🎙️ Mode Audio"):
            with gr.Row():
                with gr.Column():
                    audio_in = gr.Audio(
                        sources=["microphone"],
                        type="numpy",
                        label="🎤 Enregistrez votre question"
                    )
                    audio_btn = gr.Button("🚀 Traiter Audio", variant="primary")
                    
                with gr.Column():
                    trans_out = gr.Textbox(label="📝 Transcription", lines=2)
                    subj_out = gr.Textbox(label="🎯 Domaine")
                    resp_out = gr.Textbox(label="💡 Réponse", lines=5)
                    audio_out = gr.Audio(label="🔊 Audio")
            
            with gr.Accordion("📚 Sources RAG", open=False):
                sources_out = gr.Markdown()
        
        # Onglet Texte (pour debug)
        with gr.Tab("💬 Mode Texte"):
            with gr.Row():
                with gr.Column():
                    text_in = gr.Textbox(
                        label="💬 Posez votre question",
                        placeholder="Ex: Comment résoudre x² + 5x + 6 = 0 ?",
                        lines=3
                    )
                    text_btn = gr.Button("🚀 Traiter Texte", variant="primary")
                    
                    gr.Examples(
                        examples=[
                            "Comment résoudre une équation du second degré ?",
                            "Qu'est-ce que la troisième loi de Newton ?",
                            "Comment conjuguer le verbe to be au présent ?"
                        ],
                        inputs=text_in
                    )
                    
                with gr.Column():
                    trans_out2 = gr.Textbox(label="📝 Question", lines=2)
                    subj_out2 = gr.Textbox(label="🎯 Domaine")
                    resp_out2 = gr.Textbox(label="💡 Réponse", lines=5)
                    audio_out2 = gr.Audio(label="🔊 Audio")
            
            with gr.Accordion("📚 Sources RAG", open=False):
                sources_out2 = gr.Markdown()
    
    # Events
    audio_btn.click(
        fn=process_audio,
        inputs=[audio_in],
        outputs=[trans_out, subj_out, resp_out, audio_out, sources_out]
    )
    
    text_btn.click(
        fn=process_text,
        inputs=[text_in],
        outputs=[trans_out2, subj_out2, resp_out2, audio_out2, sources_out2]
    )

print("✅ Interface créée!")
print("=" * 70)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🚀 Étape 9 : Lancement !
</VSCode.Cell>

<VSCode.Cell language="python">
print("🚀 Lancement de l'application...")
print("=" * 70)
print("\n📱 Un lien public va apparaître...")
print("💡 Cliquez dessus pour ouvrir l'interface!")
print("\n" + "=" * 70)

demo.launch(
    share=True,
    debug=True
)
</VSCode.Cell>
````